# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Wisconsin', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_3', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'source_target_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 12, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.55815 time= 0.11600
Epoch: 0002 train_loss= 1.26587 time= 0.00304
Epoch: 0003 train_loss= 1.07121 time= 0.00195
Epoch: 0004 train_loss= 0.94668 time= 0.00396
Epoch: 0005 train_loss= 0.91341 time= 0.00301
Epoch: 0006 train_loss= 0.90704 time= 0.00302
Epoch: 0007 train_loss= 0.88764 time= 0.00293
Epoch: 0008 train_loss= 0.85419 time= 0.00300
Epoch: 0009 train_loss= 0.85127 time= 0.00300
Epoch: 0010 train_loss= 0.83926 time= 0.00400
Epoch: 0011 train_loss= 0.81741 time= 0.00300
Epoch: 0012 train_loss= 0.79079 time= 0.00298
Epoch: 0013 train_loss= 0.76027 time= 0.00301
Epoch: 0014 train_loss= 0.73331 time= 0.00201
Epoch: 0015 train_loss= 0.70903 time= 0.00300
Epoch: 0016 train_loss= 0.69621 time= 0.00396
Epoch: 0017 train_loss= 0.68394 time= 0.00197
Epoch: 0018 train_loss= 0.65087 time= 0.00199
Epoch: 0019 train_loss= 0.65190 time= 0

Epoch: 0189 train_loss= 0.29590 time= 0.00400
Epoch: 0190 train_loss= 0.28633 time= 0.00300
Epoch: 0191 train_loss= 0.29447 time= 0.00300
Epoch: 0192 train_loss= 0.28927 time= 0.00400
Epoch: 0193 train_loss= 0.28582 time= 0.00300
Epoch: 0194 train_loss= 0.28484 time= 0.00300
Epoch: 0195 train_loss= 0.28599 time= 0.00301
Epoch: 0196 train_loss= 0.28800 time= 0.00299
Epoch: 0197 train_loss= 0.28866 time= 0.00301
Epoch: 0198 train_loss= 0.28426 time= 0.00300
Epoch: 0199 train_loss= 0.28420 time= 0.00299
Epoch: 0200 train_loss= 0.30090 time= 0.00301
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.54354 time= 0.08001
Epoch: 0002 train_loss= 1.20250 time= 0.00302
Epoch: 0003 train_loss= 1.11177 time= 0.00499
Epoch: 0004 train_loss= 1.01689 time= 0.00300
Epoch: 0005 train_loss= 0.90993 time= 0.00299
Epoch: 0006 train_loss= 0.90859 time= 0.00302
Epoch: 0007 train_loss= 0.91710 time= 

Epoch: 0198 train_loss= 0.29035 time= 0.00399
Epoch: 0199 train_loss= 0.29807 time= 0.00404
Epoch: 0200 train_loss= 0.29220 time= 0.00405
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.58550 time= 0.09701
Epoch: 0002 train_loss= 1.21679 time= 0.00302
Epoch: 0003 train_loss= 1.00833 time= 0.00397
Epoch: 0004 train_loss= 0.95734 time= 0.00195
Epoch: 0005 train_loss= 0.94749 time= 0.00400
Epoch: 0006 train_loss= 0.99749 time= 0.00401
Epoch: 0007 train_loss= 0.94319 time= 0.00299
Epoch: 0008 train_loss= 0.94975 time= 0.00194
Epoch: 0009 train_loss= 0.89901 time= 0.00200
Epoch: 0010 train_loss= 0.88914 time= 0.00304
Epoch: 0011 train_loss= 0.87212 time= 0.00302
Epoch: 0012 train_loss= 0.83356 time= 0.00298
Epoch: 0013 train_loss= 0.80619 time= 0.00313
Epoch: 0014 train_loss= 0.78450 time= 0.00201
Epoch: 0015 train_loss= 0.77679 time= 0.00305
Epoch: 0016 train_loss= 0.76707 time= 

Epoch: 0189 train_loss= 0.29933 time= 0.00396
Epoch: 0190 train_loss= 0.29658 time= 0.00499
Epoch: 0191 train_loss= 0.29612 time= 0.00300
Epoch: 0192 train_loss= 0.30047 time= 0.00401
Epoch: 0193 train_loss= 0.30820 time= 0.00400
Epoch: 0194 train_loss= 0.31329 time= 0.00400
Epoch: 0195 train_loss= 0.30364 time= 0.00289
Epoch: 0196 train_loss= 0.29908 time= 0.00300
Epoch: 0197 train_loss= 0.30348 time= 0.00269
Epoch: 0198 train_loss= 0.30121 time= 0.00300
Epoch: 0199 train_loss= 0.30666 time= 0.00299
Epoch: 0200 train_loss= 0.30777 time= 0.00311
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.52652 time= 0.10355
Epoch: 0002 train_loss= 1.23928 time= 0.00503
Epoch: 0003 train_loss= 1.02137 time= 0.00401
Epoch: 0004 train_loss= 0.94387 time= 0.00199
Epoch: 0005 train_loss= 0.94964 time= 0.00402
Epoch: 0006 train_loss= 0.93440 time= 0.00298
Epoch: 0007 train_loss= 0.94545 time= 

Epoch: 0195 train_loss= 0.29246 time= 0.00400
Epoch: 0196 train_loss= 0.29274 time= 0.00300
Epoch: 0197 train_loss= 0.29050 time= 0.00200
Epoch: 0198 train_loss= 0.29193 time= 0.00301
Epoch: 0199 train_loss= 0.29210 time= 0.00300
Epoch: 0200 train_loss= 0.29282 time= 0.00304
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.45374 time= 0.10600
Epoch: 0002 train_loss= 1.33098 time= 0.00306
Epoch: 0003 train_loss= 1.15624 time= 0.00294
Epoch: 0004 train_loss= 1.04372 time= 0.00300
Epoch: 0005 train_loss= 0.96231 time= 0.00300
Epoch: 0006 train_loss= 0.93206 time= 0.00194
Epoch: 0007 train_loss= 0.90683 time= 0.00297
Epoch: 0008 train_loss= 0.91561 time= 0.00201
Epoch: 0009 train_loss= 0.89272 time= 0.00301
Epoch: 0010 train_loss= 0.88256 time= 0.00299
Epoch: 0011 train_loss= 0.86872 time= 0.00299
Epoch: 0012 train_loss= 0.84285 time= 0.00300
Epoch: 0013 train_loss= 0.80894 time= 

...

Epoch: 0186 train_loss= 0.29567 time= 0.00400
Epoch: 0187 train_loss= 0.29056 time= 0.00307
Epoch: 0188 train_loss= 0.28834 time= 0.00300
Epoch: 0189 train_loss= 0.28884 time= 0.00352
Epoch: 0190 train_loss= 0.28899 time= 0.00342
Epoch: 0191 train_loss= 0.28794 time= 0.00301
Epoch: 0192 train_loss= 0.29328 time= 0.00201
Epoch: 0193 train_loss= 0.29034 time= 0.00199
Epoch: 0194 train_loss= 0.29118 time= 0.00299
Epoch: 0195 train_loss= 0.28928 time= 0.00294
Epoch: 0196 train_loss= 0.29219 time= 0.00301
Epoch: 0197 train_loss= 0.29129 time= 0.00300
Epoch: 0198 train_loss= 0.29677 time= 0.00293
Epoch: 0199 train_loss= 0.28954 time= 0.00305
Epoch: 0200 train_loss= 0.28598 time= 0.00292
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.60914 time= 0.12516
Epoch: 0002 train_loss= 1.21481 time= 0.00300
Epoch: 0003 train_loss= 1.12148 time= 0.00296
Epoch: 0004 train_loss= 1.00742 time= 

Epoch: 0182 train_loss= 0.29930 time= 0.00508
Epoch: 0183 train_loss= 0.29674 time= 0.00292
Epoch: 0184 train_loss= 0.29480 time= 0.00195
Epoch: 0185 train_loss= 0.29576 time= 0.00200
Epoch: 0186 train_loss= 0.29674 time= 0.00299
Epoch: 0187 train_loss= 0.30221 time= 0.00300
Epoch: 0188 train_loss= 0.30148 time= 0.00310
Epoch: 0189 train_loss= 0.28900 time= 0.00290
Epoch: 0190 train_loss= 0.29670 time= 0.00318
Epoch: 0191 train_loss= 0.28791 time= 0.00287
Epoch: 0192 train_loss= 0.30059 time= 0.00296
Epoch: 0193 train_loss= 0.28971 time= 0.00303
Epoch: 0194 train_loss= 0.29156 time= 0.00197
Epoch: 0195 train_loss= 0.29427 time= 0.00313
Epoch: 0196 train_loss= 0.29770 time= 0.00395
Epoch: 0197 train_loss= 0.29789 time= 0.00294
Epoch: 0198 train_loss= 0.29601 time= 0.00304
Epoch: 0199 train_loss= 0.28744 time= 0.00297
Epoch: 0200 train_loss= 0.29499 time= 0.00298
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Trai

Epoch: 0179 train_loss= 0.29355 time= 0.00198
Epoch: 0180 train_loss= 0.29794 time= 0.00300
Epoch: 0181 train_loss= 0.29437 time= 0.00408
Epoch: 0182 train_loss= 0.29703 time= 0.00300
Epoch: 0183 train_loss= 0.28915 time= 0.00300
Epoch: 0184 train_loss= 0.29945 time= 0.00296
Epoch: 0185 train_loss= 0.30070 time= 0.00307
Epoch: 0186 train_loss= 0.28999 time= 0.00293
Epoch: 0187 train_loss= 0.30375 time= 0.00400
Epoch: 0188 train_loss= 0.29844 time= 0.00304
Epoch: 0189 train_loss= 0.30645 time= 0.00302
Epoch: 0190 train_loss= 0.29658 time= 0.00294
Epoch: 0191 train_loss= 0.30166 time= 0.00305
Epoch: 0192 train_loss= 0.29291 time= 0.00295
Epoch: 0193 train_loss= 0.29640 time= 0.00306
Epoch: 0194 train_loss= 0.29754 time= 0.00393
Epoch: 0195 train_loss= 0.29928 time= 0.00308
Epoch: 0196 train_loss= 0.29580 time= 0.00397
Epoch: 0197 train_loss= 0.29499 time= 0.00496
Epoch: 0198 train_loss= 0.29079 time= 0.00312
Epoch: 0199 train_loss= 0.28530 time= 0.00291
Epoch: 0200 train_loss= 0.28691 ti

Epoch: 0168 train_loss= 0.30409 time= 0.00300
Epoch: 0169 train_loss= 0.29970 time= 0.00392
Epoch: 0170 train_loss= 0.30319 time= 0.00391
Epoch: 0171 train_loss= 0.30543 time= 0.00299
Epoch: 0172 train_loss= 0.30460 time= 0.00200
Epoch: 0173 train_loss= 0.30765 time= 0.00337
Epoch: 0174 train_loss= 0.30199 time= 0.00304
Epoch: 0175 train_loss= 0.30305 time= 0.00297
Epoch: 0176 train_loss= 0.30157 time= 0.00398
Epoch: 0177 train_loss= 0.29723 time= 0.00298
Epoch: 0178 train_loss= 0.29998 time= 0.00307
Epoch: 0179 train_loss= 0.29767 time= 0.00296
Epoch: 0180 train_loss= 0.29785 time= 0.00311
Epoch: 0181 train_loss= 0.30149 time= 0.00202
Epoch: 0182 train_loss= 0.29682 time= 0.00295
Epoch: 0183 train_loss= 0.29469 time= 0.00309
Epoch: 0184 train_loss= 0.29505 time= 0.00397
Epoch: 0185 train_loss= 0.29427 time= 0.00295
Epoch: 0186 train_loss= 0.29402 time= 0.00317
Epoch: 0187 train_loss= 0.29908 time= 0.00302
Epoch: 0188 train_loss= 0.29136 time= 0.00392
Epoch: 0189 train_loss= 0.29424 ti

Epoch: 0173 train_loss= 0.30409 time= 0.00400
Epoch: 0174 train_loss= 0.30721 time= 0.00300
Epoch: 0175 train_loss= 0.29889 time= 0.00296
Epoch: 0176 train_loss= 0.30055 time= 0.00300
Epoch: 0177 train_loss= 0.30540 time= 0.00299
Epoch: 0178 train_loss= 0.30382 time= 0.00407
Epoch: 0179 train_loss= 0.30572 time= 0.00301
Epoch: 0180 train_loss= 0.30808 time= 0.00294
Epoch: 0181 train_loss= 0.29299 time= 0.00403
Epoch: 0182 train_loss= 0.29760 time= 0.00302
Epoch: 0183 train_loss= 0.30597 time= 0.00294
Epoch: 0184 train_loss= 0.29606 time= 0.00404
Epoch: 0185 train_loss= 0.29602 time= 0.00297
Epoch: 0186 train_loss= 0.29591 time= 0.00195
Epoch: 0187 train_loss= 0.29782 time= 0.00298
Epoch: 0188 train_loss= 0.29468 time= 0.00307
Epoch: 0189 train_loss= 0.29510 time= 0.00294
Epoch: 0190 train_loss= 0.29991 time= 0.00405
Epoch: 0191 train_loss= 0.29962 time= 0.00300
Epoch: 0192 train_loss= 0.29983 time= 0.00402
Epoch: 0193 train_loss= 0.30491 time= 0.00396
Epoch: 0194 train_loss= 0.29093 ti